# User Songs!

A brief diversion into how exactly the user songs are represented.

In [1]:
cd -q '..'

In [2]:
import mido

In [3]:
from commons.messages import controlstate

I've recorded one track (track 3) on one user song (user song 2).

In [6]:
!python extractor.py documents/data/user_song_tests/1track.syx -S 2

               Recorded   Duration       Size
 User Song 2       True         41       2112
     Track A      False          0         99
     Track 1      False          0          0
     Track 2      False          0          0
     Track 3       True         41       1999
     Track 4      False          0          0
     Track 5      False          0          0



What's in the MIDI?

In [8]:
!python extractor.py documents/data/user_song_tests/1track.syx -s 2 -n documents/data/user_song_tests/1track_{}.mid

In [9]:
mf = mido.MidiFile('documents/data/user_song_tests/1track_2.mid')

In [10]:
mf

<midi file 'documents/data/user_song_tests/1track_2.mid' type 1, 2 tracks, 499 messages>

In [13]:
mf.tracks

[<midi track '' 10 messages>, <midi track '' 489 messages>]

We have the time track, which we put first, and then all the other tracks.

In [17]:
mf.print_tracks()

=== Track 0
<meta message time_signature numerator=3 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message set_tempo tempo=400000 time=0>
<message sysex data=(126,127,9,1) time=0>
<message sysex data=(67,21,76,2,1,0,1,0) time=0>
<message sysex data=(67,21,76,2,1,32,66,17) time=0>
<meta message text text=' YAMAHA ' time=0>
<meta message sequencer_specific data=(67, 118, 26, 4, 0, 0) time=0>
<meta message sequencer_specific data=(67, 118, 26, 2, 100) time=0>
<meta message sequencer_specific data=(67, 118, 26, 1, 0) time=0>
<meta message end_of_track time=0>
=== Track 1
<message sysex data=(67,18,76,2,1,0,1,0) time=0>
<message sysex data=(67,18,76,2,1,32,66,17) time=0>
<meta message text text=' YAMAHA ' time=0>
<message control_change channel=2 control=0 value=0 time=0>
<message control_change channel=2 control=32 value=112 time=0>
<message program_change channel=2 program=0 time=0>
<message control_change channel=12 control=0 value=0 time=0>
<message contr

In [18]:
mido.tempo2bpm(400000)

150.0

In [4]:
def hexspace(x):
    return " ".join(format(b, "02X") for b in x)

def trackprint(track):
    s = controlstate.MidiControlState()
    t = 0
    for m in track:
        t += m.time
        if m.is_meta:
            if m.type == 'sequencer_specific':
                print(t, 'Sequencer Specific', hexspace(m.data))
            else:
                print(t, m)
        else:
            w = s.feed(m)
            if w:
                print(t, w)
            else:
                print(t, m)


In [37]:
trackprint(mf.tracks[0])

0 <meta message time_signature numerator=3 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=400000 time=0>
0 GM System ON
0 Reverb Type 01(Hall1)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 00
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 00
0 <meta message end_of_track time=0>


In the time track, we have the time signature and tempo, along with reverb and chorus type. There's a meta message for YAMAHA text, and some sequencer specific messages; all this happens at time 0.

In [38]:
trackprint(mf.tracks[1])

0 Reverb Type 01(Hall1)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 2 Bank MSB 0
0 2 Bank LSB 112
0 2 Program Change [0,112,0] 003 Grand Piano (PIANO)
0 C Bank MSB 0
0 C Bank LSB 112
0 C Program Change [0,112,85] 103 Voice Lead (SYNTH LEAD)
0 2 RPN MSB 0
0 2 RPN LSB 0
0 2 Data Entry MSB: Pitch Bend Range 2
0 2 RPN LSB 127
0 2 RPN MSB 127
0 C RPN MSB 0
0 C RPN LSB 0
0 C Data Entry MSB: Pitch Bend Range 2
0 C RPN LSB 127
0 C RPN MSB 127
0 2 Voice Reverb Level 20
0 2 Voice Chorus Level 41
0 2 [Control 94] 0
0 2 Voice Volume 100
0 2 Voice Pan 54
0 2 Expression 127
0 2 Release Time 0
0 2 Pedal Sustain OFF
0 C Voice Reverb Level 60
0 C Voice Chorus Level 24
0 C [Control 94] 0
0 C Voice Volume 100
0 C Voice Pan 74
0 C Expression 127
0 C Release Time 0
0 C Pedal Sustain OFF
0 polytouch channel=2 note=0 value=63 time=0
0 2 Pitch Bend 0
0 C Pitch Bend 0
0 2 Note 036(C1) 77
0 C Note 036(C1) 77
281 2 Note 036(C1) 0
281 C Note 036(C1) 0
285 2 Note 043(G1) 82
285 C Note 0

I recorded Track 3, with main voice 003 and dual voice 103. It seems the main voice is on Channel 0x2 and the dual voice on channel 0xC (so, Channels 3 and 13).

At the beginning, the controls are set for reverb and chorus type, pitch bend, reverb and chorus levels, voice volume, pan, expression, release time (panel sustain), as wekk as Control 94, which according to the MIDI spec table is Effects 4 (Celeste) depth, which is set to zero. The DGX-505 does not apparently recognise it or emit it, but it's something that's recorded (for compatibility?) Same with the polyphonic aftertouch.



Now, what's sent over the MIDI on Song Out playback?

In [40]:
!python slurp.py -gp DGX > documents/data/user_song_tests/1track_2.txt

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'. CtrlC to stop
^C
Stopping on KeyboardInterrupt


In [45]:
with open('documents/data/user_song_tests/1track_2.txt') as infile:
    played = list(mido_util.readin_strings(infile))

In [5]:
from commons import mido_util

In [6]:
def mprint(seq, tpb=None, bpm=None):
    s = controlstate.MidiControlState()
    rt = 0
    if tpb:
        tempo = mido.bpm2tempo(bpm)
        
    for m in seq:
        if m.type != 'clock':
            if m.type == 'start':
                rt = m.time
            t = m.time - rt
            w = s.feed(m)
            
            if tpb:
                t = mido.second2tick(t, tpb, tempo)
            
            if w:
                print(format(t, '.2f'), w)
            else:
                print(format(t, '.2f'), m)


In [69]:
mprint(played, 96, 150)

0.00 start time=2.2532950180002445
2.66 5 All Sound OFF
2.88 5 Note 024(C0) 77
283.93 5 Note 024(C0) 0
287.76 5 Note 031(G0) 82
565.68 5 Note 026(D0) 65
568.79 5 Note 031(G0) 0
842.62 5 Note 026(D0) 0
846.70 5 Note 029(F0) 68
1145.74 5 Note 029(F0) 0
1147.66 5 Note 019(G-1) 70
1238.61 5 Note 059(B2) 65
1329.82 5 Note 059(B2) 0
1331.73 5 Note 062(D3) 72
1420.53 5 Note 029(F0) 56
1421.01 5 Note 062(D3) 0
1422.69 5 Note 069(A3) 71
1438.77 5 Note 019(G-1) 0
1508.85 5 Note 067(G3) 63
1518.69 5 Note 069(A3) 0
1596.71 5 Note 065(F3) 75
1609.66 5 Note 067(G3) 0
1687.64 5 Note 024(C0) 55
1692.69 5 Note 064(E3) 79
1698.68 5 Note 029(F0) 0
1701.56 5 Note 065(F3) 0
1920.68 5 Note 064(E3) 0
2006.84 5 Note 024(C0) 0
2291.71 5 Note 029(F0) 65
2384.60 5 Note 065(F3) 65
2473.63 5 Note 065(F3) 0
2478.66 5 Note 069(A3) 70
2546.82 5 Note 069(A3) 0
2576.82 5 Note 081(A4) 61
2666.82 5 Note 081(A4) 0
2667.78 5 Note 079(G4) 66
2752.74 5 Note 029(F0) 0
2757.78 5 Note 077(F4) 63
2764.74 5 Note 079(G4) 0
2834.82

In [50]:
!python slurp.py -ngp DGX > documents/data/user_song_tests/1track_ini.txt

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'. CtrlC to stop
^C
Stopping on KeyboardInterrupt


In [51]:
with open('documents/data/user_song_tests/1track_ini.txt') as infile:
    ini1 = list(mido_util.readin_strings(infile))

In [70]:
mprint(ini1, 96, 150)

574.68 GM System ON
622.90 XG System ON
670.91 MIDI Master Tuning 0
671.85 Reverb Type 01(Hall1)
672.57 Chorus Type 1(Chorus1)
694.41 3 Bank MSB 0
694.43 3 Bank LSB 0
694.64 3 Program Change [0,0,0] 134 Grand Piano (XG PIANO)
694.88 3 RPN MSB 0
695.12 3 RPN LSB 0
695.14 3 Data Entry MSB: Pitch Bend Range 2
695.36 3 RPN LSB 127
695.38 3 RPN MSB 127
695.60 3 Voice Reverb Level 40
695.84 3 Voice Chorus Level 0
696.02 3 Voice Volume 100
696.03 3 Voice Pan 64
696.32 3 Pedal Sustain OFF
696.34 3 Modulation Wheel 0
696.56 3 Expression 127
696.80 3 Harmonic Content 0
696.82 3 Release Time 0
697.04 3 Attack Time 0
697.28 3 Brightness 0
697.52 3 Pitch Bend 0
697.76 4 Bank MSB 0
697.78 4 Bank LSB 0
698.00 4 Program Change [0,0,0] 134 Grand Piano (XG PIANO)
698.25 4 RPN MSB 0
698.26 4 RPN LSB 0
698.48 4 Data Entry MSB: Pitch Bend Range 2
698.72 4 RPN LSB 127
698.74 4 RPN MSB 127
698.96 4 Voice Reverb Level 40
699.20 4 Voice Chorus Level 0
699.22 4 Voice Volume 100
699.44 4 Voice Pan 64
699.68 4 Pe

Interesting things to note:

The SONG midi output only includes the Main voice, on channel `5`. The Dual voice is not present (I suspect this is because the is no more room. `0` through `2` are keyboard voice, then `3` through `7` are for song tracks, and `8` through `F` are for style and accompaniment.

The track was recorded with MAIN OCTAVE set to -1, DUAL OCTAVE set to 0. This is reflected in the output from the SONG out, which are shifted one octave down, but not for the data stored in the MID file... This data must be *somewhere*... are they repurposing the polyphonic aftertouch??

If this is indeed the case, then for proper export we must actually parse the messages to correct for this!



In [71]:
!python collect.py -g DGX > documents/data/user_song_tests/2track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [72]:
!python extractor.py documents/data/user_song_tests/2track.syx -S 2 -s 2 -n documents/data/user_song_tests/2track_{}.mid

               Recorded   Duration       Size
 User Song 2       True         35       3805
     Track A      False          0         99
     Track 1      False          0          0
     Track 2      False          0          0
     Track 3       True         31       1999
     Track 4      False          0          0
     Track 5       True         35       1693



In [73]:
!python slurp.py -ngp DGX > documents/data/user_song_tests/2track_ini.txt

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'. CtrlC to stop
^C
Stopping on KeyboardInterrupt


In [75]:
with open('documents/data/user_song_tests/2track_ini.txt') as infile:
    ini2 = list(mido_util.readin_strings(infile))
mprint(ini2)

5.37 GM System ON
5.57 XG System ON
5.77 MIDI Master Tuning 0
5.77 Reverb Type 01(Hall1)
5.78 Chorus Type 1(Chorus1)
5.87 3 Bank MSB 0
5.87 3 Bank LSB 0
5.87 3 Program Change [0,0,0] 134 Grand Piano (XG PIANO)
5.87 3 RPN MSB 0
5.87 3 RPN LSB 0
5.87 3 Data Entry MSB: Pitch Bend Range 2
5.87 3 RPN LSB 127
5.87 3 RPN MSB 127
5.87 3 Voice Reverb Level 40
5.87 3 Voice Chorus Level 0
5.87 3 Voice Volume 100
5.88 3 Voice Pan 64
5.88 3 Pedal Sustain OFF
5.88 3 Modulation Wheel 0
5.88 3 Expression 127
5.88 3 Harmonic Content 0
5.88 3 Release Time 0
5.88 3 Attack Time 0
5.88 3 Brightness 0
5.88 3 Pitch Bend 0
5.88 4 Bank MSB 0
5.88 4 Bank LSB 0
5.88 4 Program Change [0,0,0] 134 Grand Piano (XG PIANO)
5.88 4 RPN MSB 0
5.88 4 RPN LSB 0
5.89 4 Data Entry MSB: Pitch Bend Range 2
5.89 4 RPN LSB 127
5.89 4 RPN MSB 127
5.89 4 Voice Reverb Level 40
5.89 4 Voice Chorus Level 0
5.89 4 Voice Volume 100
5.89 4 Voice Pan 64
5.89 4 Pedal Sustain OFF
5.89 4 Modulation Wheel 0
5.89 4 Expression 127
5.89 4 Harmo

Recorded another track, this time with a different tempo.
The Main voice was voice 205 with octave -2, the dual voice was voice 005 with octave +2.

In [76]:
mf2 = mido.MidiFile('documents/data/user_song_tests/2track_2.mid')

In [79]:
trackprint(mf2.tracks[0])

0 <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=500000 time=0>
0 GM System ON
0 Reverb Type 01(Hall1)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 00
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 00
0 <meta message end_of_track time=0>


The tempo is overwritten,  and Whoops, I apparently overwrote the time signature as well.

In [81]:
mf2.tracks[1] == mf.tracks[1]

True

In [82]:
trackprint(mf2.tracks[2])

0 Reverb Type 01(Hall1)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 4 Bank MSB 0
0 4 Bank LSB 0
0 4 Program Change [0,0,22] 205 Harmonica (XG ORGAN)
0 E Bank MSB 0
0 E Bank LSB 112
0 E Program Change [0,112,3] 005 Honky-tonk Piano (PIANO)
0 4 RPN MSB 0
0 4 RPN LSB 0
0 4 Data Entry MSB: Pitch Bend Range 2
0 4 RPN LSB 127
0 4 RPN MSB 127
0 E RPN MSB 0
0 E RPN LSB 0
0 E Data Entry MSB: Pitch Bend Range 2
0 E RPN LSB 127
0 E RPN MSB 127
0 4 Voice Reverb Level 28
0 4 Voice Chorus Level 0
0 4 [Control 94] 0
0 4 Voice Volume 104
0 4 Voice Pan 64
0 4 Expression 127
0 4 Release Time 0
0 4 Pedal Sustain OFF
0 E Voice Reverb Level 32
0 E Voice Chorus Level 0
0 E [Control 94] 0
0 E Voice Volume 65
0 E Voice Pan 64
0 E Expression 127
0 E Release Time 0
0 E Pedal Sustain OFF
0 polytouch channel=4 note=0 value=62 time=0
0 4 Pitch Bend 0
0 E Pitch Bend 0
0 4 Note 060(C3) 50
0 E Note 084(C5) 50
0 4 Note 072(C4) 51
0 E Note 096(C6) 51
7 4 Pedal Sustain ON
7 E Pedal Sustain ON

The Main voice was recorded onto channel `4` (5), and dual voice onto channel `E` (15).

The Dual notes are recorded with +2 octave on them, but the main voice is not. Instead, it's got a polytouch message with value 62. Hmmm.

What happens if we try to change the voice while recording?

We can't use the function menus, but what about scrolling the voice.
Do the defaults get set?

What about using values saved in the bank registers?

What happens if we change the reverb and chorus type?

What about transpose?


In [91]:
!python extractor.py documents/data/user_song_tests/2track.syx -R 2,2 3,2

Bank 2, Button 2:
       Style number: 050
      Accompaniment: OFF
           Main A/B: Main B
              Tempo: 082
  Main Voice number: 078
  Dual Voice number: 076
 Split Voice number: 046
            Harmony:  ON
               Dual:  ON
              Split: OFF
       Style Volume: 100
          Transpose:  00
   Pitch Bend Range:  02
        Split Point: 054(F#2)
          M. Volume: 103
          M. Octave:   0
             M. Pan: 064
    M. Reverb Level: 040
    M. Chorus Level: 000
          D. Volume: 066
          D. Octave:  -1
             D. Pan: 064
    D. Reverb Level: 040
    D. Chorus Level: 020
          S. Volume: 100
          S. Octave:   0
             S. Pan: 064
    S. Reverb Level: 008
    S. Chorus Level: 000
        Reverb Type: 03(Hall3)
        Chorus Type: 1(Chorus1)
            Sustain: OFF
       Harmony Type: 01(Duet)
     Harmony Volume: 080

Bank 3, Button 2:
       Style number: 092
      Accompaniment:  ON
           Main A/B: Main A
         

|Main Voice                          |M Volume|M Octave|M Reverb Lv|M Chorus Lv|Dual Voice                     |D Volume|D Octave|D Reverb Lv|D Chorus Lv|
|:-----------------------------------|-------:|-------:|----------:|----------:|:------------------------------|-------:|-------:|----------:|----------:|
|085 Trombone Section                |     110|      -1|         34|          0|088 Brass Section              |      90|      -1|         32|          0|
|086 French Horn                     |      80|       0|         40|          0|084 Trombone                   |      70|       0|         36|          0|
|087 Tuba                            |     127|      -2|         18|          0|045 Acoustic Bass              |      84|      -2|          0|          0|
|119 Music Box                       |     100|      +1|         20|          0|108 SweetHeaven                |      60|       0|         42|          0|
|075 Baritone Sax                    |     112|      -2|         28|          0|088 Brass Section              |     100|      -1|         24|          0|
|025 16'+2' Organ                    |      70|      -1|         28|         34|026 16'+4' Organ               |      30|      +1|         28|          0|


Let's record track 4, starting with the chorus and reverb types set to something different. Then, cycle through voices 085, 086, 087, 119, 075, 025, with dual ON.



In [93]:
!python collect.py -g DGX > documents/data/user_song_tests/3track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [94]:
!python extractor.py documents/data/user_song_tests/3track.syx -S 2 -s 2 -n documents/data/user_song_tests/3track_{}.mid

               Recorded   Duration       Size
 User Song 2       True        143       5578
     Track A      False          0         99
     Track 1      False          0          0
     Track 2      False          0          0
     Track 3       True        127       1999
     Track 4       True        133       1773
     Track 5       True        143       1693



In [95]:
mf3 = mido.MidiFile('documents/data/user_song_tests/3track_2.mid')

In [96]:
mf3.tracks[1] == mf2.tracks[1]

True

In [97]:
mf3.tracks[3] == mf2.tracks[2]

True

In [99]:
trackprint(mf3.tracks[0])

0 <meta message time_signature numerator=1 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=454528 time=0>
0 GM System ON
0 Reverb Type 08(Plate1)
0 Chorus Type 4(Flanger2)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 00
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 00
0 <meta message end_of_track time=0>


The reverb and chorus type got overwritten. Also, I tried setting the time signature to 0, and it's at 1/4, which makes sense.

In [103]:
trackprint(mf3.tracks[2])

0 Reverb Type 08(Plate1)
0 Chorus Type 4(Flanger2)
0 <meta message text text=' YAMAHA ' time=0>
0 3 Bank MSB 0
0 3 Bank LSB 112
0 3 Program Change [0,112,62] 091 Synth Brass (BRASS)
0 D Bank MSB 0
0 D Bank LSB 112
0 D Program Change [0,112,79] 100 Ocarina (FLUTE)
0 3 RPN MSB 0
0 3 RPN LSB 0
0 3 Data Entry MSB: Pitch Bend Range 2
0 3 RPN LSB 127
0 3 RPN MSB 127
0 D RPN MSB 0
0 D RPN LSB 0
0 D Data Entry MSB: Pitch Bend Range 2
0 D RPN LSB 127
0 D RPN MSB 127
0 3 Voice Reverb Level 32
0 3 Voice Chorus Level 54
0 3 [Control 94] 0
0 3 Voice Volume 100
0 3 Voice Pan 43
0 3 Expression 127
0 3 Release Time 0
0 3 Pedal Sustain OFF
0 D Voice Reverb Level 87
0 D Voice Chorus Level 80
0 D [Control 94] 0
0 D Voice Volume 40
0 D Voice Pan 86
0 D Expression 127
0 D Release Time 0
0 D Pedal Sustain OFF
0 polytouch channel=3 note=0 value=64 time=0
0 3 Pitch Bend 0
0 D Pitch Bend 0
0 3 Note 053(F2) 39
0 D Note 041(F1) 39
0 3 Note 072(C4) 45
0 D Note 060(C3) 45
0 3 Note 077(F4) 42
0 D Note 065(F3) 42
0 

Okay, it definitely looks like the default settings get set. The Dual octave affects the dual notes directly, while the Main octave is signalled by a polytouch message, with offset from 64. The program changes etc. are recorded for both main and dual, even if dual is not activated. Transpose is completely ignored.

Question: Does the polytouch message work for songs in the flash memory?

What happens if we send a GM_ON or set the reverb in the middle? Does that reset the reverb and chorus types? How about regular reverb chorus messages?

Also, it seems that the durations are indeed in measures, and affected by the time signature on the time track.

Does setting LOCAL have any effect?

In [104]:
o = mido.open_output('DGX-505 MIDI 1')

In [7]:
from commons.messages import controls
import time

In [112]:
def test():
    o.send(controls.reverb_type(00, 00))
    time.sleep(2)
    o.send(controls.chorus_type(00, 00))
    time.sleep(4)
    o.send(controls.gm_on())
    time.sleep(2)
    o.send(controls.local(False))
    time.sleep(5)
    o.send(controls.local(True))
    time.sleep(3)
    o.send(controls.master_tune_val(100))
    time.sleep(3)
    o.send(controls.xg_reset())



In [113]:
test()

In [114]:
!python collect.py -g DGX > documents/data/user_song_tests/4track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [115]:
!python extractor.py documents/data/user_song_tests/4track.syx -S 2 -s 2 -n documents/data/user_song_tests/4track_{}.mid

               Recorded   Duration       Size
 User Song 2       True          9       7524
     Track A      False          0         99
     Track 1      False          0          0
     Track 2       True          6       1946
     Track 3       True          8       1999
     Track 4       True          8       1773
     Track 5       True          9       1693



In [116]:
o.close()

In [18]:
mf4 = mido.MidiFile('documents/data/user_song_tests/4track_2.mid')

In [120]:
mf4.tracks[2:] == mf3.tracks[1:]

True

In [121]:
trackprint(mf4.tracks[0])

0 <meta message time_signature numerator=15 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=600000 time=0>
0 GM System ON
0 Reverb Type 09(Plate2)
0 Chorus Type 3(Flanger1)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 00
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 00
0 <meta message end_of_track time=0>


In [122]:
trackprint(mf4.tracks[1])

0 Reverb Type 09(Plate2)
0 Chorus Type 3(Flanger1)
0 <meta message text text=' YAMAHA ' time=0>
0 1 Bank MSB 0
0 1 Bank LSB 113
0 1 Program Change [0,113,0] 001 Live! Grand Piano (PIANO)
0 B Bank MSB 0
0 B Bank LSB 112
0 B Program Change [0,112,48] 054 String Ensemble (STRINGS)
0 1 RPN MSB 0
0 1 RPN LSB 0
0 1 Data Entry MSB: Pitch Bend Range 5
0 1 RPN LSB 127
0 1 RPN MSB 127
0 B RPN MSB 0
0 B RPN LSB 0
0 B Data Entry MSB: Pitch Bend Range 5
0 B RPN LSB 127
0 B RPN MSB 127
0 1 Voice Reverb Level 20
0 1 Voice Chorus Level 0
0 1 [Control 94] 0
0 1 Voice Volume 110
0 1 Voice Pan 64
0 1 Expression 127
0 1 Release Time 46
0 1 Pedal Sustain ON
0 B Voice Reverb Level 50
0 B Voice Chorus Level 0
0 B [Control 94] 0
0 B Voice Volume 36
0 B Voice Pan 64
0 B Expression 127
0 B Release Time 46
0 B Pedal Sustain ON
0 polytouch channel=1 note=0 value=66 time=0
0 1 Pitch Bend 8190
0 B Pitch Bend 8190
0 1 Note 048(C2) 67
63 1 Pitch Bend 7540
63 B Pitch Bend 7540
65 1 Pitch Bend 7020
65 B Pitch Bend 7020

The reverb and chorus type don't get changed while the thing is recording. LOCAL and so on do not seem to have an effect on what is recorded

In [123]:
with open('documents/data/user_song_tests/4track_rec.txt') as infile:
    recd = list(mido_util.readin_strings(infile))

In [128]:
mprint(recd, 96, 100)

2729.09 0 Pitch Bend 260
2729.23 1 Pitch Bend 260
2729.39 2 Pitch Bend 260
2729.88 0 Pitch Bend 780
2730.03 1 Pitch Bend 780
2730.19 2 Pitch Bend 780
2731.18 0 Pitch Bend 1300
2731.20 1 Pitch Bend 1300
2731.33 2 Pitch Bend 1300
2732.11 0 Pitch Bend 1820
2732.14 1 Pitch Bend 1820
2732.15 2 Pitch Bend 1820
2732.75 0 Pitch Bend 2470
2732.91 1 Pitch Bend 2470
2732.92 2 Pitch Bend 2470
2733.57 0 Pitch Bend 2990
2733.74 1 Pitch Bend 2990
2733.75 2 Pitch Bend 2990
2734.35 0 Pitch Bend 3250
2734.51 1 Pitch Bend 3250
2734.52 2 Pitch Bend 3250
2735.16 0 Pitch Bend 3770
2735.31 1 Pitch Bend 3770
2735.47 2 Pitch Bend 3770
2735.98 0 Pitch Bend 4290
2736.13 1 Pitch Bend 4290
2736.29 2 Pitch Bend 4290
2736.77 0 Pitch Bend 4550
2736.92 1 Pitch Bend 4550
2737.13 2 Pitch Bend 4550
2737.56 0 Pitch Bend 5070
2737.71 1 Pitch Bend 5070
2737.87 2 Pitch Bend 5070
2738.19 0 Pitch Bend 5330
2738.35 1 Pitch Bend 5330
2738.52 2 Pitch Bend 5330
2739.01 0 Pitch Bend 5850
2739.15 1 Pitch Bend 5850
2739.31 2 Pitch Be

4940.22 0 Note 098(D6) 0
4945.65 0 Note 091(G5) 0
4945.97 0 Note 089(F5) 0
4964.54 5 Note 067(G3) 56
4971.61 5 Note 072(C4) 0
4996.71 0 Note 091(G5) 38
4996.85 0 Note 100(E6) 40
4997.01 0 Note 089(F5) 53
5015.57 0 Note 101(F6) 0
5017.82 0 Note 086(D5) 0
5022.13 0 Note 089(F5) 0
5024.05 0 Note 091(G5) 0
5069.65 5 Note 060(C3) 71
5071.59 5 Note 067(G3) 0
5073.49 0 Note 079(G4) 39
5075.67 0 Note 095(B5) 39
5086.29 0 Note 100(E6) 0
5136.23 0 Note 098(D6) 43
5138.13 0 Note 089(F5) 37
5140.12 0 Note 086(D5) 37
5143.41 0 Note 083(B4) 30
5153.65 0 Note 095(B5) 0
5158.61 7 Note 053(F2) 55
5158.62 7 Note 041(F1) 52
5158.93 6 Note 060(C3) 53
5161.65 7 Pedal Sustain OFF
5162.29 0 Note 086(D5) 0
5163.41 0 Note 089(F5) 0
5173.65 5 Note 029(F0) 61
5176.55 5 Note 069(A3) 79
5177.01 5 Note 024(C0) 0
5177.49 5 Note 060(C3) 0
5192.55 7 Pedal Sustain ON
5202.45 0 Note 083(B4) 0
5221.97 0 Note 100(E6) 59
5222.45 0 Note 089(F5) 38
5223.09 0 Note 083(B4) 27
5225.18 0 Note 086(D5) 43
5231.89 0 Note 098(D6) 0


8871.17 5 Bank MSB 0
8871.18 5 Bank LSB 112
8871.33 5 Program Change [0,112,0] 003 Grand Piano (PIANO)
8871.49 5 RPN MSB 0
8871.65 5 RPN LSB 0
8871.81 5 Data Entry MSB: Pitch Bend Range 2
8871.82 5 RPN LSB 127
8871.97 5 RPN MSB 127
8872.13 5 Voice Reverb Level 20
8872.29 5 Voice Chorus Level 41
8872.45 5 Voice Pan 54
8872.46 5 Release Time 0
8873.25 5 Pitch Bend 0
8873.73 4 Bank MSB 0
8873.74 4 Bank LSB 113
8873.89 4 Program Change [0,113,0] 001 Live! Grand Piano (PIANO)
8874.05 4 RPN MSB 0
8874.21 4 RPN LSB 0
8874.38 4 Data Entry MSB: Pitch Bend Range 5
8874.39 4 RPN LSB 127
8874.60 4 RPN MSB 127
8874.70 4 Voice Reverb Level 20
8874.86 4 Voice Chorus Level 0
8875.01 4 Voice Pan 64
8875.03 4 Release Time 46
8875.65 4 Pitch Bend 8190
8877.89 Reverb Type 09(Plate2)
8878.37 Chorus Type 3(Flanger1)


Questions: How is the Harmony handled?

How does the Octave work with drum kits?

Now recording both a style and track 1 at the same time. It's gonna sound terrible, but we aren't listening to this for fun.

Initial settings:

- Style no: 102 (6/8 March)
- Pattern: Intro A - I'll be changing this several times: Intro A - Main A - Fill B - Main B - Ending - Ending rit. Fill A, Main A, Ending, and then trying to prolong the ending by switching styles (Style switches don't get recorded, do they?)

- Style volume: 30

- Reverb Type: 06 Stage1
- Chorus Type: 2 Chorus2

Initial Voice settings

- Harmony: ON, 
- Harmony Type: Trio
- Harmony Volume: 96 

- M. Octave: -1
- D. Octave: 0

- D. Voice: 007 CP-80
- M.Voice: 301 Warm Trumpet





Okay, so it seems they don't let you change the style while recording. Makes sense.

In [14]:
!python collect.py -g DGX > documents/data/user_song_tests/5track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [15]:
!python extractor.py documents/data/user_song_tests/5track.syx -S 2 -s 2 -n documents/data/user_song_tests/5track_{}.mid

               Recorded   Duration       Size
 User Song 2       True         23       9245
     Track A       True         23        422
     Track 1       True         23       1398
     Track 2       True          6       1946
     Track 3       True          8       1999
     Track 4       True          8       1773
     Track 5       True          9       1693



In [16]:
mf5 = mido.MidiFile('documents/data/user_song_tests/5track_2.mid')

In [19]:
mf5.tracks[2:] == mf4.tracks[1:]

True

In [21]:
trackprint(mf5.tracks[1])

0 Reverb Type 06(Stage1)
0 Chorus Type 2(Chorus2)
0 <meta message text text=' YAMAHA ' time=0>
0 0 Bank MSB 0
0 0 Bank LSB 32
0 0 Program Change [0,32,56] 301 Warm Trumpet (XG BRASS)
0 A Bank MSB 0
0 A Bank LSB 113
0 A Program Change [0,113,2] 007 CP 80 (PIANO)
0 0 RPN MSB 0
0 0 RPN LSB 0
0 0 Data Entry MSB: Pitch Bend Range 2
0 0 RPN LSB 127
0 0 RPN MSB 127
0 A RPN MSB 0
0 A RPN LSB 0
0 A Data Entry MSB: Pitch Bend Range 2
0 A RPN LSB 127
0 A RPN MSB 127
0 0 Voice Reverb Level 54
0 0 Voice Chorus Level 44
0 0 [Control 94] 0
0 0 Voice Volume 104
0 0 Voice Pan 64
0 0 Expression 127
0 0 Release Time 0
0 0 Pedal Sustain OFF
0 A Voice Reverb Level 32
0 A Voice Chorus Level 0
0 A [Control 94] 0
0 A Voice Volume 70
0 A Voice Pan 64
0 A Expression 127
0 A Release Time 0
0 A Pedal Sustain OFF
0 polytouch channel=0 note=0 value=63 time=0
0 0 Pitch Bend 0
0 A Pitch Bend 0
1715 A Note 072(C4) 76
1715 0 Note 067(G3) 57
1715 0 Note 064(E3) 57
1715 0 Note 072(C4) 76
1796 A Note 072(C4) 0
1796 0 Note

In [29]:
with open('documents/data/user_song_tests/5track_rec.txt') as infile:
    in5 = list(mido_util.readin_strings(infile))

In [39]:
def channelize(msgs, ch=frozenset([0, 1, 2])):
    for m in msgs:
        try:
            c = m.channel
        except AttributeError:
            yield m
        else:
            if c in ch:
                yield m
                
def cprint(msgs, ch=frozenset([0, 1, 2]), *args, **kwargs):
    return mprint(channelize(msgs, ch), *args, **kwargs)

In [41]:
cprint(in5, {0,1,2}, 96, 100)

627.21 Reverb Type 09(Plate2)
627.67 Chorus Type 3(Flanger1)
640.00 Reverb Type 09(Plate2)
640.47 Chorus Type 3(Flanger1)
2394.22 Chorus Type 3(Flanger1)
5214.82 Chorus Type 4(Flanger2)
5263.30 Chorus Type 5(Off)
5334.34 Chorus Type 4(Flanger2)
5375.62 Chorus Type 3(Flanger1)
5414.01 Chorus Type 2(Chorus2)
5460.10 Chorus Type 1(Chorus1)
5527.14 Chorus Type 2(Chorus2)
5823.62 Reverb Type 08(Plate1)
5861.53 Reverb Type 07(Stage2)
5923.61 Reverb Type 06(Stage1)
0.00 start time=92.88281124500008
1717.10 1 Note 072(C4) 76
1717.24 0 Note 055(G2) 57
1717.41 0 Note 052(E2) 57
1717.42 0 Note 060(C3) 76
1797.88 1 Note 072(C4) 0
1798.04 0 Note 060(C3) 0
1798.05 0 Note 055(G2) 0
1798.20 0 Note 052(E2) 0
1834.95 1 Note 072(C4) 93
1835.01 0 Note 055(G2) 69
1835.16 0 Note 052(E2) 69
1835.37 0 Note 060(C3) 93
1898.53 1 Note 072(C4) 0
1898.68 0 Note 060(C3) 0
1898.69 0 Note 055(G2) 0
1898.84 0 Note 052(E2) 0
1899.00 1 Note 076(E4) 89
1899.38 0 Note 060(C3) 66
1899.39 0 Note 055(G2) 66
1899.49 0 Note 06

We have the Main Voice on channel `0` (1) and the dual voice on channel `A` (11). The Harmony voice is also on channel `0`, with the same (non-)offset octave as the main voice. This means the notes are hardcoded, and will survive getting the chords changed on them. (I suspect something similar works with the Performance Assistant for this.)
The Harmony Volume is a simple reduction of velocity, with it seems like applying Main velocity*(Harmony Volume/127) or something like it.

Interesting to note in the captured MIDI output, there are a few Portamento Control events for the harmony notes, I think when the chords changed.

In [42]:
o = mido.open_output('DGX-505 MIDI 1')

In [46]:
o.send(mido.Message('note_off', channel=0, note=60))

In [47]:
def pulse(o=o, t=0.5, n=60, v=100, c=0):
    o.send(mido.Message('note_on', channel=c, note=n, velocity=v))
    time.sleep(t)
    o.send(mido.Message('note_on', channel=c, note=n, velocity=0))
    time.sleep(t)

In [57]:
pulse(o)
o.send(mido.Message('polytouch', note=0, value=63, channel=0))
pulse(o)

Sending polytouch does nothing.

How about the chords and part information?

In [59]:
trackprint(mf5.tracks[0])

0 <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=600000 time=0>
0 GM System ON
0 Reverb Type 06(Stage1)
0 Chorus Type 2(Chorus2)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 65
0 Sequencer Specific 43 76 1A 02 1E
0 Sequencer Specific 43 76 1A 01 03
0 Sequencer Specific 43 76 1A 03 31 00 31 00
1920 Sequencer Specific 43 76 1A 01 00
2031 Sequencer Specific 43 76 1A 03 33 00 33 00
2380 Sequencer Specific 43 76 1A 01 02
2591 Sequencer Specific 43 76 1A 03 34 00 34 00
2688 Sequencer Specific 43 76 1A 01 05
2884 Sequencer Specific 43 76 1A 03 31 00 31 00
3456 Sequencer Specific 43 76 1A 01 09
3471 Sequencer Specific 43 76 1A 03 34 00 34 00
3884 Sequencer Specific 43 76 1A 01 06
4224 Sequencer Specific 43 76 1A 01 00
4680 Sequencer Specific 43 76 1A 03 31 00 31 00
5194 Sequencer Specific 43 76 1A 01 02
5376 Sequencer Specific 43 76 1A 01 05
5807 Sequencer Speci

The chords go like this:
C, E, F, C, F, C, F, C, D.

Unlike the chords in the provided CD MIDI songs, these use sequencer specific meta messages instead of SysEx for the chords. The basic format seems to be the same, but using the extended-root system so C=`31`, D=`32`, E=`33`, F=`34` and so on.

The sequencer specific messages are `43 76 1A 03 RR TT RR TT`, which is that of the PSR-225.

The equivalent SysEx, as found in the CD MIDI, is `43 7E 02 RR TT RR TT`.

The style part starts with three messages

    0 Sequencer Specific 43 76 1A 04 00 65
    0 Sequencer Specific 43 76 1A 02 1E
    0 Sequencer Specific 43 76 1A 01 03

In all previous recordings they had different values:

    0 Sequencer Specific 43 76 1A 04 00 00
    0 Sequencer Specific 43 76 1A 02 64
    0 Sequencer Specific 43 76 1A 01 00
    


In [64]:
0x65

101

In [65]:
0x1E

30

So `43 76 1A 04 xx xx` is the style number, `43 76 1A 02 xx` is the style volume.

`43 76 1A 01 xx` is a style part, which is set by default to be `00` (Main A?)

The style parts were recorded as thus:
Intro A, Main A, Fill B, Main B, Ending (B), Fill A, Main A, Fill B, Ending (B), Ending rit.(B), Fill A, Main A, Ending (A).


In [71]:
trackprint(m for m in mf5.tracks[0] if m.type == 'sequencer_specific' and m.data[3] == 1 or m.type=='set_tempo')

0 <meta message set_tempo tempo=600000 time=0>
0 Sequencer Specific 43 76 1A 01 03
1920 Sequencer Specific 43 76 1A 01 00
2269 Sequencer Specific 43 76 1A 01 02
2366 Sequencer Specific 43 76 1A 01 05
2938 Sequencer Specific 43 76 1A 01 09
3351 Sequencer Specific 43 76 1A 01 06
3691 Sequencer Specific 43 76 1A 01 00
4205 Sequencer Specific 43 76 1A 01 02
4387 Sequencer Specific 43 76 1A 01 05
4724 Sequencer Specific 43 76 1A 01 09
4898 <meta message set_tempo tempo=612224 time=174>
4946 <meta message set_tempo tempo=624992 time=48>
4994 <meta message set_tempo tempo=638272 time=48>
5042 <meta message set_tempo tempo=652160 time=48>
5090 <meta message set_tempo tempo=666656 time=48>
5138 <meta message set_tempo tempo=681792 time=48>
5187 <meta message set_tempo tempo=697664 time=49>
5224 Sequencer Specific 43 76 1A 01 06
5234 <meta message set_tempo tempo=714272 time=10>
5282 <meta message set_tempo tempo=731680 time=48>
5330 <meta message set_tempo tempo=749984 time=48>
5378 <meta messa

Intro A = `03`, Main A = `00`, Fill (A→)B = `02`, Main B = `05`, Ending B = `09`, Fill (B→)A = `06`, Ending A = `04`

This matches the PSR, except that we don't have fills AA and BB. AB and BA only.

Note that Ending A and B are separate, even though the display just says 'ENDING'.
Ending rit. isn't a special mode, it's just Ending with rit. appied through tempo.

What about the ones we encountered in the CD MIDI?

    <meta message sequencer_specific data=(67, 123, 12, 1, 2) time=0>
    <meta message sequencer_specific data=(67, 123, 0, 88, 70, 48, 50, 0, 27) time=0>

In [63]:
print(hexspace((67, 123, 12, 1, 2)))
print(hexspace((67, 123, 0, 88, 70, 48, 50, 0, 27)))

43 7B 0C 01 02
43 7B 00 58 46 30 32 00 1B


No idea what these are.

What happens when we clear out the style tempo track?


In [73]:
!python collect.py -g DGX > documents/data/user_song_tests/6track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [74]:
!python extractor.py documents/data/user_song_tests/6track.syx -S 2 -s 2 -n documents/data/user_song_tests/6track_{}.mid

               Recorded   Duration       Size
 User Song 2       True         23       8922
     Track A      False          0         99
     Track 1       True         23       1398
     Track 2       True          6       1946
     Track 3       True          8       1999
     Track 4       True          8       1773
     Track 5       True          9       1693



In [75]:
trackprint(mido.MidiFile('documents/data/user_song_tests/6track_2.mid').tracks[0])

0 <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=600000 time=0>
0 GM System ON
0 Reverb Type 06(Stage1)
0 Chorus Type 2(Chorus2)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 65
0 Sequencer Specific 43 76 1A 02 1E
0 Sequencer Specific 43 76 1A 01 00
0 <meta message end_of_track time=0>


Looks like it erased all the chords a nd so on, but not the tempo. It even left the style number and volume intact.


Let's see if we can get the other status messages.
Start with Intro B, and maybe we try to get Fill AA somehow? Also set the style number > 127, to overflow into the previous byte,

In [76]:
!python collect.py -g DGX > documents/data/user_song_tests/7track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [77]:
!python extractor.py documents/data/user_song_tests/7track.syx -S 2 -s 2 -n documents/data/user_song_tests/7track_{}.mid

               Recorded   Duration       Size
 User Song 2       True         32       9103
     Track A       True         32        280
     Track 1       True         23       1398
     Track 2       True          6       1946
     Track 3       True          8       1999
     Track 4       True          8       1773
     Track 5       True          9       1693



In [78]:
trackprint(mido.MidiFile('documents/data/user_song_tests/7track_2.mid').tracks[0])

0 <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=214272 time=0>
0 GM System ON
0 Reverb Type 06(Stage1)
0 Chorus Type 2(Chorus2)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 84
0 Sequencer Specific 43 76 1A 02 4D
0 Sequencer Specific 43 76 1A 01 08
0 Sequencer Specific 43 76 1A 03 45 00 45 00
1536 Sequencer Specific 43 76 1A 01 05
2688 Sequencer Specific 43 76 1A 01 00
3072 Sequencer Specific 43 76 1A 01 05
3456 Sequencer Specific 43 76 1A 01 00
3840 Sequencer Specific 43 76 1A 01 05
4224 Sequencer Specific 43 76 1A 01 00
4608 Sequencer Specific 43 76 1A 01 05
4992 Sequencer Specific 43 76 1A 01 00
5376 Sequencer Specific 43 76 1A 01 05
6528 Sequencer Specific 43 76 1A 01 00
7296 Sequencer Specific 43 76 1A 01 04
7680 Sequencer Specific 43 76 1A 01 05
8448 Sequencer Specific 43 76 1A 01 09
8832 Sequencer Specific 43 76 1A 01 00
9264 Sequencer Specific 43

It turns out the piano accompaniments don't have a fill. Oh well.
Also, meta messages can handle values >127.

So here we have confirmation that Intro B is `08`.

Now, to double-check the ticks per beat, and also see when the duration overflows.

Let's record User Song 1, but with time signature 1/4 and tempo 280 bpm.

With harmony on Tremolo 1/4, so that the notes should turn on every beat, record >256 measures.
Also while we're at it try to check drum kit octaves, maybe.

In [80]:
!python collect.py -g DGX > documents/data/user_song_tests/8track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [81]:
!python extractor.py documents/data/user_song_tests/8track.syx -S 1 -s 1 -n documents/data/user_song_tests/8track_{}.mid

               Recorded   Duration       Size
 User Song 1       True        344       1620
     Track A      False          0         99
     Track 1       True        344       1507
     Track 2      False          0          0
     Track 3      False          0          0
     Track 4      False          0          0
     Track 5      False          0          0



In [82]:
t0 = mido.MidiFile('documents/data/user_song_tests/8track_1.mid')

In [83]:
t0

<midi file 'documents/data/user_song_tests/8track_1.mid' type 1, 2 tracks, 472 messages>

In [84]:
trackprint(t0.tracks[0])

0 <meta message time_signature numerator=1 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=214272 time=0>
0 GM System ON
0 Reverb Type 02(Hall2)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 00
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 00
0 <meta message end_of_track time=0>


In [85]:
trackprint(t0.tracks[1])

0 Reverb Type 02(Hall2)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 0 Bank MSB 0
0 0 Bank LSB 113
0 0 Program Change [0,113,0] 001 Live! Grand Piano (PIANO)
0 A Bank MSB 0
0 A Bank LSB 112
0 A Program Change [0,112,48] 054 String Ensemble (STRINGS)
0 0 RPN MSB 0
0 0 RPN LSB 0
0 0 Data Entry MSB: Pitch Bend Range 2
0 0 RPN LSB 127
0 0 RPN MSB 127
0 A RPN MSB 0
0 A RPN LSB 0
0 A Data Entry MSB: Pitch Bend Range 2
0 A RPN LSB 127
0 A RPN MSB 127
0 0 Voice Reverb Level 20
0 0 Voice Chorus Level 0
0 0 [Control 94] 0
0 0 Voice Volume 110
0 0 Voice Pan 64
0 0 Expression 127
0 0 Release Time 0
0 0 Pedal Sustain OFF
0 A Voice Reverb Level 50
0 A Voice Chorus Level 0
0 A [Control 94] 0
0 A Voice Volume 36
0 A Voice Pan 64
0 A Expression 127
0 A Release Time 0
0 A Pedal Sustain OFF
0 polytouch channel=0 note=0 value=64 time=0
0 0 Pitch Bend 0
0 A Pitch Bend 0
0 0 Note 072(C4) 80
47 0 Note 072(C4) 0
95 0 Note 072(C4) 80
143 0 Note 072(C4) 0
191 0 Note 072(C4) 80
239 0 N

In [97]:
trackprint(t0.tracks[1][38:384])

0 0 Note 072(C4) 80
47 0 Note 072(C4) 0
95 0 Note 072(C4) 80
143 0 Note 072(C4) 0
191 0 Note 072(C4) 80
239 0 Note 072(C4) 0
287 0 Note 072(C4) 80
335 0 Note 072(C4) 0
383 0 Note 072(C4) 80
431 0 Note 072(C4) 0
479 0 Note 072(C4) 80
527 0 Note 072(C4) 0
575 0 Note 072(C4) 80
623 0 Note 072(C4) 0
671 0 Note 072(C4) 80
719 0 Note 072(C4) 0
767 0 Note 072(C4) 80
815 0 Note 072(C4) 0
863 0 Note 072(C4) 80
911 0 Note 072(C4) 0
959 0 Note 072(C4) 80
1007 0 Note 072(C4) 0
1055 0 Note 072(C4) 80
1103 0 Note 072(C4) 0
1151 0 Note 072(C4) 80
1199 0 Note 072(C4) 0
1247 0 Note 072(C4) 80
1295 0 Note 072(C4) 0
1343 0 Note 072(C4) 80
1391 0 Note 072(C4) 0
1439 0 Note 072(C4) 80
1487 0 Note 072(C4) 0
1535 0 Note 072(C4) 80
1583 0 Note 072(C4) 0
1631 0 Note 072(C4) 80
1679 0 Note 072(C4) 0
1727 0 Note 072(C4) 80
1775 0 Note 072(C4) 0
1823 0 Note 072(C4) 80
1871 0 Note 072(C4) 0
1919 0 Note 072(C4) 80
1967 0 Note 072(C4) 0
2015 0 Note 072(C4) 80
2063 0 Note 072(C4) 0
2111 0 Note 072(C4) 80
2159 0 Note 

In [98]:
pulse_messages = t0.tracks[1][38:384]

In [103]:
from collections import Counter

In [104]:
Counter(m.time for m in pulse_messages)

Counter({0: 1, 47: 1, 48: 344})

There are 48 ticks between each note-on and note-off (excepting some weirdness at the start), which gives 48×2=96 ticks between each note_on, or 96 ticks per beat.

Changing to the registry voice with everything the same except the octave sends out the octave change (polytouch) message, as well as pitch bend range changes for both main and dual voices.

And the drum kit does behave like any other voice with respect to the octave offset (in this case, -1); the messages recorded are not the actual notes played (eg. if a midi player or using mido-play to send back over MIDI to the keyboard). I wonder if this will be recognised for flash memory MIDI; I know that the style number meta-messages are not but the chord changes are, from an older experiment.





The next experiment is to play every chord. I attempted to play every chord that appears in the manual, as well as the weird one that isn't, and the non-chord that happens if you mash keys. Fun!

In [105]:
!python collect.py -g DGX > documents/data/user_song_tests/9track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [106]:
!python extractor.py documents/data/user_song_tests/9track.syx -S 1 -s 1 -n documents/data/user_song_tests/9track_{}.mid

               Recorded   Duration       Size
 User Song 1       True        275       7186
     Track A       True        223       5665
     Track 1       True        275       1507
     Track 2      False          0          0
     Track 3      False          0          0
     Track 4      False          0          0
     Track 5      False          0          0



In [9]:
t1 = mido.MidiFile('documents/data/user_song_tests/9track_1.mid')

In [110]:
t0.tracks[1] == t1.tracks[1]

True

In [111]:
trackprint(t1.tracks[0])

0 <meta message time_signature numerator=5 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=624992 time=0>
0 GM System ON
0 Reverb Type 02(Hall2)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 36
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 05
481 Sequencer Specific 43 76 1A 01 06
631 Sequencer Specific 43 76 1A 01 02
728 Sequencer Specific 43 76 1A 01 06
828 Sequencer Specific 43 76 1A 01 02
960 Sequencer Specific 43 76 1A 01 06
1047 Sequencer Specific 43 76 1A 01 02
1440 Sequencer Specific 43 76 1A 01 05
1926 Sequencer Specific 43 76 1A 01 06
2400 Sequencer Specific 43 76 1A 01 00
2491 Sequencer Specific 43 76 1A 03 31 00 31 00
4314 Sequencer Specific 43 76 1A 03 31 04 31 04
4792 Sequencer Specific 43 76 1A 03 31 01 31 01
5754 Sequencer Specific 43 76 1A 03 31 04 31 04
5757 Sequencer Specific 43 76 1A 03 31 06 31 06
6715 Sequencer Specific 43 76 1A

Note: mashing the A/B button will not allow you to access FILL AA. I guess we don't support it.

In [112]:
tmetas = [m for m in t1.tracks[0] if m.type == 'sequencer_specific']

In [122]:
for i, m in enumerate(tmetas):
    print("{:3d}: {:4d} | {:<14} #".format(i, m.time, hexspace(m.data[3:])))
    

  0:    0 | 04 00 36       #
  1:    0 | 02 64          #
  2:    0 | 01 05          #
  3:  481 | 01 06          #
  4:  150 | 01 02          #
  5:   97 | 01 06          #
  6:  100 | 01 02          #
  7:  132 | 01 06          #
  8:   87 | 01 02          #
  9:  393 | 01 05          #
 10:  486 | 01 06          #
 11:  474 | 01 00          #
 12:   91 | 03 31 00 31 00 #
 13: 1823 | 03 31 04 31 04 #
 14:  478 | 03 31 01 31 01 #
 15:  962 | 03 31 04 31 04 #
 16:    3 | 03 31 06 31 06 #
 17:  958 | 03 31 02 31 02 #
 18:  957 | 03 31 05 31 05 #
 19:  961 | 03 31 03 31 03 #
 20:  953 | 03 44 27 31 1E #
 21:  968 | 03 44 29 31 1E #
 22: 1441 | 03 31 24 31 24 #
 23:  960 | 03 31 23 31 23 #
 24:  959 | 03 31 20 31 20 #
 25: 1116 | 03 31 07 31 07 #
 26:  807 | 03 31 1C 31 1C #
 27:  955 | 03 31 08 31 08 #
 28:  658 | 03 31 0C 31 0C #
 29:  787 | 03 31 09 31 09 #
 30:  962 | 03 31 0A 31 0A #
 31:  961 | 03 31 0D 31 0D #
 32: 1435 | 03 23 21 23 21 #
 33:    3 | 03 31 0E 31 0E #
 34: 1242 | 03


Now, let's attempt to annotate. First, I tried to do all the recognised chords in the manual order, then some other things, and then all the keys in order, and then I tried to get some weird looking chords.

    Seq: delta| Bytes
    
      0:    0 | 04 00 36       # Style No.
      1:    0 | 02 64          # Style Vol
      2:    0 | 01 05          # MAIN B
      3:  481 | 01 06          # FILL A
      4:  150 | 01 02          # FILL B
      5:   97 | 01 06          # FILL A
      6:  100 | 01 02          # FILL B
      7:  132 | 01 06          # FILL A
      8:   87 | 01 02          # FILL B
      9:  393 | 01 05          # MAIN B
     10:  486 | 01 06          # FILL A
     11:  474 | 01 00          # MAIN A
     12:   91 | 03 31 00 31 00 # C Major
     13: 1823 | 03 31 04 31 04 # C(9)
     14:  478 | 03 31 01 31 01 # C6
     15:  962 | 03 31 04 31 04 # (intermittent C9)
     16:    3 | 03 31 06 31 06 # C6(9) 
     17:  958 | 03 31 02 31 02 # CM7
     18:  957 | 03 31 05 31 05 # CM7(9)
     19:  961 | 03 31 03 31 03 # CM7(#11)
     20:  953 | 03 44 27 31 1E # (Forgot to press E and ended up with some F# chords)
     21:  968 | 03 44 29 31 1E #
     22: 1441 | 03 31 24 31 24 # Cb5
     23:  960 | 03 31 23 31 23 # CM7b5
     24:  959 | 03 31 20 31 20 # Csus4
     25: 1116 | 03 31 07 31 07 # Caug
     26:  807 | 03 31 1C 31 1C # CM7aug
     27:  955 | 03 31 08 31 08 # Cm
     28:  658 | 03 31 0C 31 0C # Cm(9)
     29:  787 | 03 31 09 31 09 # Cm6
     30:  962 | 03 31 0A 31 0A # Cm7
     31:  961 | 03 31 0D 31 0D # Cm7(9)
     32: 1435 | 03 23 21 23 21 # (intermittent chord)
     33:    3 | 03 31 0E 31 0E # Cm7(11)
     34: 1242 | 03 31 0F 31 0F # CmM7
     35:  857 | 03 23 00 23 00 # (intermittent chord)
     36:    2 | 03 31 10 31 10 # CmM7(9)
     37: 1252 | 03 31 0B 31 0B # Cm7b5
     38:  965 | 03 31 25 31 25 # CmM7b5
     39: 1917 | 03 31 11 31 11 # Cdim
     40:  480 | 03 31 12 31 12 # Cdim7
     41:  483 | 03 31 13 31 13 # C7
     42:  957 | 03 31 19 31 19 # C7(b9)
     43:  963 | 03 31 1A 31 1A # C7(b13)
     44:  969 | 03 31 16 31 16 # C7(9)
     45: 1913 | 03 31 17 31 17 # C7(#11)
     46:  959 | 03 31 18 31 18 # C7(13)
     47: 1122 | 03 31 19 31 19 # C7(b9) again
     48:  791 | 03 31 1B 31 1B # C7(#9)
     49: 1337 | 03 31 13 31 13 # (intermittent)
     50:    3 | 03 31 15 31 15 # C7b5
     51: 1060 | 03 31 1D 31 1D # C7aug
     52:  482 | 03 31 14 31 14 # C7sus4
     53:  960 | 03 31 21 31 21 # Csus2
     54:  963 | 03 31 1F 31 1F # C1+5
     55:  957 | 03 31 1E 31 1E # C1+8
     56: 2097 | 03 31 26 31 26 # C* (C + Db + Eb)
     57:  788 | 03 22 00 22 00 # (intermittent)
     58:    3 | 03 31 27 31 27 # C* (C + Db + 
     59:  958 | 03 31 28 31 28 # C* (C + Db + 
     60:  586 | 01 02          #
     61:  376 | 01 05          #
     62:    4 | 03 31 00 31 00 # C
     63:  460 | 03 22 00 22 00 # Db
     64:  489 | 03 32 00 32 00 # D
     65:  482 | 03 23 00 23 00 # Eb
     66:  479 | 03 33 00 33 00 # E
     67:  486 | 03 34 00 34 00 # F
     68:   99 | 03 44 00 44 00 # F#
     69:   96 | 03 35 00 35 00 # G
     70:   95 | 03 45 00 45 00 # G#
     71:  190 | 03 36 00 36 00 # A
     72:  104 | 03 27 00 27 00 # Bb
     73:  178 | 03 37 00 37 00 # B
     74:  191 | 03 31 00 31 00 # C
     75:  549 | 03 31 1E 31 1E # C1+8
     76:  224 | 03 37 1E 37 1E # B1+8
     77:  194 | 03 27 1E 27 1E # B♭1+8
     78:  282 | 03 36 1E 36 1E # A1+8
     79:  186 | 03 45 1E 45 1E # G♯1+8
     80:  288 | 03 35 00 35 00 # (intermittent G)
     81:    3 | 03 35 1E 35 1E # G1+8
     82:  194 | 03 44 00 44 00 # (intermittent F#)
     83:    3 | 03 44 1E 44 1E # F♯1+8
     84:  278 | 03 34 1E 34 1E # F1+8
     85:  205 | 03 33 00 33 00 # (intermittent E)
     86:    4 | 03 33 1E 33 1E # E1+8
     87:  276 | 03 23 00 23 00 # (intermittent Eb)
     88:    2 | 03 23 1E 23 1E # Eb1+8
     89:  191 | 03 32 1E 32 1E # D1+8
     90:  285 | 03 22 1E 22 1E # Db1+8
     91:  200 | 03 31 1E 31 1E # C1+8
     92:  485 | 03 31 00 35 1E # C/G  (you get this with a fourth)
     93:  467 | 03 22 00 45 1E # Db/G#
     94:  291 | 03 32 00 36 1E # D/A
     95:  194 | 03 23 00 27 1E # Eb/Bb
     96:  289 | 03 33 00 37 1E # E/B
     97:  191 | 03 34 00 34 00 # (intermittent F)
     98:    2 | 03 34 00 31 1E # F/C
     99:  285 | 03 44 00 22 1E # F#/Db
    100:  193 | 03 35 00 35 00 # (intermittent G)
    101:    3 | 03 35 00 32 1E # G/D
    102:  275 | 03 45 00 23 1E # G#/Eb
    103:  194 | 03 36 00 36 00 # (intermittent A)
    104:    3 | 03 36 00 33 1E # A/E
    105:  287 | 03 27 00 27 00 # (intermittent Bb)
    106:    3 | 03 27 00 34 1E # Bb/F
    107:  196 | 03 37 00 44 1E # B/F#
    108:  286 | 03 31 00 35 1E # C/G
    109:  191 | 03 33 08 33 08 # (intermittent Em)
    110:    2 | 03 31 00 31 00 # C  (C alone)
    111:  482 | 03 31 13 31 13 # C7 (C with adjacent B shortcut)
    112:  280 | 03 31 08 31 08 # Cm (C with adjacent Bb shortcut)
    113:  197 | 03 36 08 36 08 # Am (A + C)
    114:  285 | 03 45 00 45 00 # G# (G#+C)
    115:  198 | 03 31 00 35 1E # C/G (G + C)
    116:  272 | 03 44 24 44 24 # F#b5 (F# + C)
    117:  206 | 03 34 1F 34 1F # F1+5 (F + C)
    118:  291 | 03 31 00 33 1E # C/E (E + C)
    119:  184 | 03 32 13 32 13 # D7 
    120:  479 | 03 31 04 31 04 # C9 
    121:  388 | 03 32 00 32 00 # (intermittent D)
    122:    3 | 03 31 04 31 04 # C9
    123:   87 | 03 32 00 32 00 # D
    124:  109 | 03 31 04 31 04 #

    296:    3 | 03 23 26 34 1E #
    297:  960 | 03 33 00 33 00 #
    298:    3 | 03 23 22 23 22 # The Mysterious Chord 22
    299:    2 | 03 33 0F 34 1E #
    
    306:  195 | 03 34 00 34 00 #
    307:    4 | 03 23 28 34 1E # Chord 28
    308:  146 | 03 34 00 34 00 #
   
    336:    3 | 03 45 23 33 1E #
    337:  175 | 03 22 00 37 1E #
    338:    2 | 03 37 22 34 1E # Chord 22, but with a bottom accompaniment
    339:  207 | 03 27 08 27 08 #
    

Time for even more testing?

In [8]:
!python collect.py -g DGX > documents/data/user_song_tests/10track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [10]:
trackprint(t1.tracks[1])

0 Reverb Type 02(Hall2)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 0 Bank MSB 0
0 0 Bank LSB 113
0 0 Program Change [0,113,0] 001 Live! Grand Piano (PIANO)
0 A Bank MSB 0
0 A Bank LSB 112
0 A Program Change [0,112,48] 054 String Ensemble (STRINGS)
0 0 RPN MSB 0
0 0 RPN LSB 0
0 0 Data Entry MSB: Pitch Bend Range 2
0 0 RPN LSB 127
0 0 RPN MSB 127
0 A RPN MSB 0
0 A RPN LSB 0
0 A Data Entry MSB: Pitch Bend Range 2
0 A RPN LSB 127
0 A RPN MSB 127
0 0 Voice Reverb Level 20
0 0 Voice Chorus Level 0
0 0 [Control 94] 0
0 0 Voice Volume 110
0 0 Voice Pan 64
0 0 Expression 127
0 0 Release Time 0
0 0 Pedal Sustain OFF
0 A Voice Reverb Level 50
0 A Voice Chorus Level 0
0 A [Control 94] 0
0 A Voice Volume 36
0 A Voice Pan 64
0 A Expression 127
0 A Release Time 0
0 A Pedal Sustain OFF
0 polytouch channel=0 note=0 value=64 time=0
0 0 Pitch Bend 0
0 A Pitch Bend 0
0 0 Note 072(C4) 80
47 0 Note 072(C4) 0
95 0 Note 072(C4) 80
143 0 Note 072(C4) 0
191 0 Note 072(C4) 80
239 0 N

In [11]:
!python extractor.py documents/data/user_song_tests/10track.syx -S 1 -s 1 -n documents/data/user_song_tests/10track_{}.mid

               Recorded   Duration       Size
 User Song 1       True        275       9133
     Track A       True        223       5665
     Track 1       True        275       1507
     Track 2       True         22       1947
     Track 3      False          0          0
     Track 4      False          0          0
     Track 5      False          0          0



In [12]:
!python extractor.py documents/data/user_song_tests/8track.syx -S 1


               Recorded   Duration       Size
 User Song 1       True        344       1620
     Track A      False          0         99
     Track 1       True        344       1507
     Track 2      False          0          0
     Track 3      False          0          0
     Track 4      False          0          0
     Track 5      False          0          0



What's going on with the durations?

344 measures in 1/4 should be ~68 measures in 5/4 right? Why is Track 1 longer than Track A?

In [13]:
tr8 = mido.MidiFile('documents/data/user_song_tests/8track_1.mid')

In [17]:
trackprint(tr8.tracks[0])

0 <meta message time_signature numerator=1 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=214272 time=0>
0 GM System ON
0 Reverb Type 02(Hall2)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 00
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 00
0 <meta message end_of_track time=0>


In [15]:
tr9 = mido.MidiFile('documents/data/user_song_tests/9track_1.mid')

In [18]:
trackprint(tr9.tracks[0])

0 <meta message time_signature numerator=5 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=624992 time=0>
0 GM System ON
0 Reverb Type 02(Hall2)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 36
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 05
481 Sequencer Specific 43 76 1A 01 06
631 Sequencer Specific 43 76 1A 01 02
728 Sequencer Specific 43 76 1A 01 06
828 Sequencer Specific 43 76 1A 01 02
960 Sequencer Specific 43 76 1A 01 06
1047 Sequencer Specific 43 76 1A 01 02
1440 Sequencer Specific 43 76 1A 01 05
1926 Sequencer Specific 43 76 1A 01 06
2400 Sequencer Specific 43 76 1A 01 00
2491 Sequencer Specific 43 76 1A 03 31 00 31 00
4314 Sequencer Specific 43 76 1A 03 31 04 31 04
4792 Sequencer Specific 43 76 1A 03 31 01 31 01
5754 Sequencer Specific 43 76 1A 03 31 04 31 04
5757 Sequencer Specific 43 76 1A 03 31 06 31 06
6715 Sequencer Specific 43 76 1A

In [20]:
trackprint(tr8.tracks[1])

0 Reverb Type 02(Hall2)
0 Chorus Type 1(Chorus1)
0 <meta message text text=' YAMAHA ' time=0>
0 0 Bank MSB 0
0 0 Bank LSB 113
0 0 Program Change [0,113,0] 001 Live! Grand Piano (PIANO)
0 A Bank MSB 0
0 A Bank LSB 112
0 A Program Change [0,112,48] 054 String Ensemble (STRINGS)
0 0 RPN MSB 0
0 0 RPN LSB 0
0 0 Data Entry MSB: Pitch Bend Range 2
0 0 RPN LSB 127
0 0 RPN MSB 127
0 A RPN MSB 0
0 A RPN LSB 0
0 A Data Entry MSB: Pitch Bend Range 2
0 A RPN LSB 127
0 A RPN MSB 127
0 0 Voice Reverb Level 20
0 0 Voice Chorus Level 0
0 0 [Control 94] 0
0 0 Voice Volume 110
0 0 Voice Pan 64
0 0 Expression 127
0 0 Release Time 0
0 0 Pedal Sustain OFF
0 A Voice Reverb Level 50
0 A Voice Chorus Level 0
0 A [Control 94] 0
0 A Voice Volume 36
0 A Voice Pan 64
0 A Expression 127
0 A Release Time 0
0 A Pedal Sustain OFF
0 polytouch channel=0 note=0 value=64 time=0
0 0 Pitch Bend 0
0 A Pitch Bend 0
0 0 Note 072(C4) 80
47 0 Note 072(C4) 0
95 0 Note 072(C4) 80
143 0 Note 072(C4) 0
191 0 Note 072(C4) 80
239 0 N

In [21]:
tr10 = mido.MidiFile('documents/data/user_song_tests/10track_1.mid')

In [27]:
tr10.tracks[0] == tr9.tracks[0]

False

In [32]:
tr10.tracks[0][5:] == tr9.tracks[0][5:]

True

In [37]:
trackprint(tr10.tracks[0][:5])

trackprint(tr9.tracks[0][:5])

0 <meta message time_signature numerator=5 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=624992 time=0>
0 GM System ON
0 Reverb Type 10(Off)
0 Chorus Type 5(Off)
0 <meta message time_signature numerator=5 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=624992 time=0>
0 GM System ON
0 Reverb Type 02(Hall2)
0 Chorus Type 1(Chorus1)


In [36]:
trackprint(tr10.tracks[2])

0 Reverb Type 10(Off)
0 Chorus Type 5(Off)
0 <meta message text text=' YAMAHA ' time=0>
0 1 Bank MSB 0
0 1 Bank LSB 113
0 1 Program Change [0,113,0] 001 Live! Grand Piano (PIANO)
0 B Bank MSB 0
0 B Bank LSB 112
0 B Program Change [0,112,48] 054 String Ensemble (STRINGS)
0 1 RPN MSB 0
0 1 RPN LSB 0
0 1 Data Entry MSB: Pitch Bend Range 2
0 1 RPN LSB 127
0 1 RPN MSB 127
0 B RPN MSB 0
0 B RPN LSB 0
0 B Data Entry MSB: Pitch Bend Range 2
0 B RPN LSB 127
0 B RPN MSB 127
0 1 Voice Reverb Level 127
0 1 Voice Chorus Level 127
0 1 [Control 94] 0
0 1 Voice Volume 110
0 1 Voice Pan 64
0 1 Expression 127
0 1 Release Time 0
0 1 Pedal Sustain OFF
0 B Voice Reverb Level 50
0 B Voice Chorus Level 0
0 B [Control 94] 0
0 B Voice Volume 36
0 B Voice Pan 64
0 B Expression 127
0 B Release Time 0
0 B Pedal Sustain OFF
0 polytouch channel=1 note=0 value=64 time=0
0 1 Pitch Bend 0
0 B Pitch Bend 0
0 1 Note 072(C4) 67
69 1 Note 076(E4) 35
72 1 Note 072(C4) 0
106 1 Note 079(G4) 62
110 1 Note 076(E4) 0
172 1 Note

I recorded Track 2 with LOCAL OFF, Performance Assistant ON melody mode, the reverb and chorus types set OFF but the reverb and chorus levels set to maximum. Local OFF did nothing but make it impossible to hear what you're playing.

The performance assistant actually affects the notes being played, so that is hard coded into the track.

Changing the reverb and chorus types actually overwrites the reverb and chorus types on track A.

Back to those durations...

In [48]:
# measures = (ticks) * (beats / tick) * (measures / beats)
#          = (ticks) / (ticks / beat) / (beats / measure)

def measures(midifile, track_no):
    ticks = sum(m.time for m in midifile.tracks[track_no])
    timesig = midifile.tracks[0][0]
    return int(ticks / midifile.ticks_per_beat / timesig.numerator)

In [49]:
measures(tr8, 1)

344

In [50]:
measures(tr8, 0)

0

In [51]:
measures(tr9, 1)

68

In [52]:
measures(tr9, 0)

224

In [54]:
[measures(tr10, i) for i in range(3)]

[224, 68, 22]

In [55]:
!python extractor.py documents/data/user_song_tests/10track.syx -S 1

               Recorded   Duration       Size
 User Song 1       True        275       9133
     Track A       True        223       5665
     Track 1       True        275       1507
     Track 2       True         22       1947
     Track 3      False          0          0
     Track 4      False          0          0
     Track 5      False          0          0



In [70]:
def measures2(midifile, track_no):
    ticks = sum(m.time for m in midifile.tracks[track_no])
    timesig = midifile.tracks[0][0]
    return (ticks / (timesig.numerator * midifile.ticks_per_beat))

In [72]:
measures2(tr10, 0)

224.0

... maybe they round up and subtract 1?

In [73]:
import struct

In [105]:
def binspace(x):
    return " ".join(format(b, "08b") for b in x)

def bprint(x, f='>L'):
    print(binspace(struct.pack('>L', x)))


In [104]:
bprint(344)
bprint(68)
bprint(275)

00000000 00000000 00000001 01011000
00000000 00000000 00000000 01000100
00000000 00000000 00000001 00010011


*Interesting*, are these really weird values...

Is it a coincidence that 275 is 68 shifted up two places with ones inserted??

In [84]:
68 << 2 | 0b11

275

I dunno. Probably.

Additional tests:
1. New track with a different tempo and chorus type
    - What about overwriting the tempo?
    - When we overwrite the Reverb and Chorus types, does it affect playback for all tracks?
2. Overwrite the style track with a new style with a different time signature
    - Can we get even weirder durations?
    - What exactly are the chord codes for the following combinations:
         - C* (C + D♭ + E♭)
         - C* (C + D♭ + F♯)
         - C* (C + D♭ + G)
         - C* (C + D♭ + B♭)
         - C* (C + D + E)
         - Weird bass only chord (C + D♭ + D + E♭)
         - Weird bass only chord with bottom note (C + D + E♭ + E) (same as above)?
         - Weird bass only chord with bottom note (G + C + D♭ + D + E♭) (same as above)?
         - Weird bass only chord with bottom note (G + C + D♭ + B) (same as above)?
         - No chord? (C + D♭ + B)
         - Transpositions of the above
         - Versions with various bottom notes added

In [85]:
!python collect.py -g DGX > documents/data/user_song_tests/11track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [86]:
!python extractor.py documents/data/user_song_tests/11track.syx -S 1 -s 1 -n documents/data/user_song_tests/11track_{}.mid

               Recorded   Duration       Size
 User Song 1       True        275       9462
     Track A       True        223       5665
     Track 1       True        275       1507
     Track 2       True         22       1947
     Track 3      False          0          0
     Track 4      False          0          0
     Track 5       True          2        329



In [99]:
tr11 = mido.MidiFile('documents/data/user_song_tests/11track_1.mid')

In [90]:
[tr10.tracks[i] == tr11.tracks[i] for i in range(3)]

[False, True, True]

In [91]:
trackprint(x for x, y in zip(tr11.tracks[0], tr10.tracks[0]) if x != y)

0 Reverb Type 09(Plate2)
0 Chorus Type 4(Flanger2)


In [92]:
trackprint(tr11.tracks[3])

0 Reverb Type 09(Plate2)
0 Chorus Type 4(Flanger2)
0 <meta message text text=' YAMAHA ' time=0>
0 4 Bank MSB 0
0 4 Bank LSB 113
0 4 Program Change [0,113,0] 001 Live! Grand Piano (PIANO)
0 E Bank MSB 0
0 E Bank LSB 112
0 E Program Change [0,112,48] 054 String Ensemble (STRINGS)
0 4 RPN MSB 0
0 4 RPN LSB 0
0 4 Data Entry MSB: Pitch Bend Range 2
0 4 RPN LSB 127
0 4 RPN MSB 127
0 E RPN MSB 0
0 E RPN LSB 0
0 E Data Entry MSB: Pitch Bend Range 2
0 E RPN LSB 127
0 E RPN MSB 127
0 4 Voice Reverb Level 20
0 4 Voice Chorus Level 0
0 4 [Control 94] 0
0 4 Voice Volume 110
0 4 Voice Pan 64
0 4 Expression 127
0 4 Release Time 0
0 4 Pedal Sustain OFF
0 E Voice Reverb Level 50
0 E Voice Chorus Level 0
0 E [Control 94] 0
0 E Voice Volume 36
0 E Voice Pan 64
0 E Expression 127
0 E Release Time 0
0 E Pedal Sustain OFF
0 polytouch channel=4 note=0 value=64 time=0
0 4 Pitch Bend 0
0 E Pitch Bend 0
0 4 Note 063(Eb3) 48
45 4 Note 063(Eb3) 0
45 4 Note 064(E3) 64
77 4 Note 064(E3) 0
120 4 Note 067(G3) 75
260 

You can't change the tempo if a style track is recorded, but the chorus and reverb types are overwritten... and yes, upon hearing the playback, it affects the whole instrument, not just per track.

Now, we overwrite the style track with a style with a different time signature and tempo, and accompany with some weird chords. 

Let's try JazzWaltz2 (115).



In [93]:
!python collect.py -g DGX > documents/data/user_song_tests/12track.syx

Reading from port 'DGX-505:DGX-505 MIDI 1 20:0'
Section: Song data
Message  1 of 39,  2048/76904 data bytes received
Message  2 of 39,  4096/76904 data bytes received
Message  3 of 39,  6144/76904 data bytes received
Message  4 of 39,  8192/76904 data bytes received
Message  5 of 39, 10240/76904 data bytes received
Message  6 of 39, 12288/76904 data bytes received
Message  7 of 39, 14336/76904 data bytes received
Message  8 of 39, 16384/76904 data bytes received
Message  9 of 39, 18432/76904 data bytes received
Message 10 of 39, 20480/76904 data bytes received
Message 11 of 39, 22528/76904 data bytes received
Message 12 of 39, 24576/76904 data bytes received
Message 13 of 39, 26624/76904 data bytes received
Message 14 of 39, 28672/76904 data bytes received
Message 15 of 39, 30720/76904 data bytes received
Message 16 of 39, 32768/76904 data bytes received
Message 17 of 39, 34816/76904 data bytes received
Message 18 of 39, 36864/76904 data bytes received
Message 19 of 39, 38912/76904 dat

In [94]:
!python extractor.py documents/data/user_song_tests/12track.syx -S 1 -s 1 -n documents/data/user_song_tests/12track_{}.mid

               Recorded   Duration       Size
 User Song 1       True        367       4763
     Track A       True        139        966
     Track 1       True        367       1507
     Track 2       True         30       1947
     Track 3      False          0          0
     Track 4      False          0          0
     Track 5       True          3        329



In [95]:
5/3 * 275

458.33333333333337

In [98]:
tr12 = mido.MidiFile('documents/data/user_song_tests/12track_1.mid')

In [100]:
[tr11.tracks[i] == tr12.tracks[i] for i in range(4)]

[False, True, True, True]

In [109]:
[measures(tr12, x) for x in range(4)]

[139, 114, 38, 4]

In [112]:
[5/3 * measures(tr11, x) for x in range(4)]

[373.33333333333337, 113.33333333333334, 36.66666666666667, 3.3333333333333335]

In [111]:
bprint(30)
bprint(38)

00000000 00000000 00000000 00011110
00000000 00000000 00000000 00100110


Yeah, I don't know what's going on with the numbers.

In [108]:
trackprint(tr12.tracks[0])

0 <meta message time_signature numerator=3 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
0 <meta message set_tempo tempo=374976 time=0>
0 GM System ON
0 Reverb Type 08(Plate1)
0 Chorus Type 3(Flanger1)
0 <meta message text text=' YAMAHA ' time=0>
0 Sequencer Specific 43 76 1A 04 00 72
0 Sequencer Specific 43 76 1A 02 64
0 Sequencer Specific 43 76 1A 01 00
0 Sequencer Specific 43 76 1A 03 31 00 31 00
2868 Sequencer Specific 43 76 1A 03 23 08 23 08
2873 Sequencer Specific 43 76 1A 03 31 26 31 26
4608 Sequencer Specific 43 76 1A 03 31 27 31 27
5816 Sequencer Specific 43 76 1A 03 31 28 31 28
6919 Sequencer Specific 43 76 1A 03 31 29 31 29
8057 Sequencer Specific 43 76 1A 03 37 22 37 22
10356 Sequencer Specific 43 76 1A 03 22 13 22 13
10363 Sequencer Specific 43 76 1A 03 22 22 31 1E
12385 Sequencer Specific 43 76 1A 03 31 00 31 00
12944 Sequencer Specific 43 76 1A 03 23 22 31 1E
13542 Sequencer Specific 43 76 1A 03 31 00 31 00
14101 Sequencer Specific 43 76 1A 03 4

In [115]:
for i, m in enumerate(m for m in tr12.tracks[0] if m.type =='sequencer_specific'):
    print("{:3d}: {:4d} | {:<14} # ".format(i, m.time, hexspace(m.data[3:])))

  0:    0 | 04 00 72       # 
  1:    0 | 02 64          # 
  2:    0 | 01 00          # 
  3:    0 | 03 31 00 31 00 # 
  4: 2868 | 03 23 08 23 08 # 
  5:    5 | 03 31 26 31 26 # 
  6: 1735 | 03 31 27 31 27 # 
  7: 1208 | 03 31 28 31 28 # 
  8: 1103 | 03 31 29 31 29 # 
  9: 1138 | 03 37 22 37 22 # 
 10: 2299 | 03 22 13 22 13 # 
 11:    7 | 03 22 22 31 1E # 
 12: 2022 | 03 31 00 31 00 # 
 13:  559 | 03 23 22 31 1E # 
 14:  598 | 03 31 00 31 00 # 
 15:  559 | 03 44 22 31 1E # 
 16:  579 | 03 31 00 31 00 # 
 17: 1741 | 03 22 22 35 1E # 
 18:  567 | 03 32 13 32 13 # 
 19:    4 | 03 22 22 44 1E # 
 20:  566 | 03 31 22 34 1E # 
 21:    4 | 03 22 22 34 1E # 
 22:  290 | 03 33 00 33 00 # 
 23:    4 | 03 22 22 33 1E # 
 24:  295 | 03 31 22 23 1E # 
 25:    4 | 03 22 22 23 1E # 
 26:  269 | 03 31 26 32 1E # 
 27:    4 | 03 22 22 32 1E # 
 28:  280 | 03 22 22 22 1E # 
 29:  293 | 03 22 22 31 1E # 
 30: 1144 | 03 37 22 37 22 # 
 31: 2318 | 03 31 26 31 26 # 
 32:  573 | 03 31 2A 31 2A # 
 33:  581 

Again, we look for the chords

      0:    0 | 04 00 72       # Style no.
      1:    0 | 02 64          # vol.
      2:    0 | 01 00          # Main A
      3:    0 | 03 31 00 31 00 # C
      4: 2868 | 03 23 08 23 08 # (int)
      5:    5 | 03 31 26 31 26 # C*: C + D♭ + E♭
      6: 1735 | 03 31 27 31 27 # C*: C + D♭ + F♯
      7: 1208 | 03 31 28 31 28 # C*: C + D♭ + G
      8: 1103 | 03 31 29 31 29 # C*: C + D♭ + B♭
      9: 1138 | 03 37 22 37 22 # no chord: C + D♭ + B: -> B-C-D♭ Chord 22 -> Bx
     10: 2299 | 03 22 13 22 13 # (int)
     11:    7 | 03 22 22 31 1E # C bass: C + D♭ + D + E♭ -> C + D♭-D-E♭ Chord 22 -> D♭x/C
     12: 2022 | 03 31 00 31 00 # C
     13:  559 | 03 23 22 31 1E # C bass: C + D + E♭ + E -> C + D-E♭-E Chord 22 -> Dx/C
     14:  598 | 03 31 00 31 00 # C
     15:  559 | 03 44 22 31 1E # C bass: F♯x/C
     16:  579 | 03 31 00 31 00 # C
     17: 1741 | 03 22 22 35 1E # G bass: D♭x/G
     18:  567 | 03 32 13 32 13 # (int)
     19:    4 | 03 22 22 44 1E # F♯ bass: D♭x/F♯
     20:  566 | 03 31 22 34 1E # (int)
     21:    4 | 03 22 22 34 1E # F bass: D♭x/F
     22:  290 | 03 33 00 33 00 # (int)
     23:    4 | 03 22 22 33 1E # E bass: D♭x/E
     24:  295 | 03 31 22 23 1E # (int)
     25:    4 | 03 22 22 23 1E # E♭ bass: D♭x/E♭
     26:  269 | 03 31 26 32 1E # (int)
     27:    4 | 03 22 22 32 1E # D bass: D♭x/D
     28:  280 | 03 22 22 22 1E # D♭ bass: "D♭x/D♭" ?
     29:  293 | 03 22 22 31 1E # C bass: C♭x/C
     30: 1144 | 03 37 22 37 22 # no chord: Bx
     31: 2318 | 03 31 26 31 26 # C*: C + D♭ + E♭
     32:  573 | 03 31 2A 31 2A # C*: C + D + E
     33:  581 | 03 31 26 31 26 # C*: C + D♭ + E♭
     34:  577 | 03 31 2A 31 2A # C*: C + D + E
     35:  577 | 03 31 26 31 26 # C*: C + D♭ + E♭
     36:  565 | 03 31 26 22 1E # C*/D♭: D♭ + C + (D♭?) + E♭
     37:  578 | 03 22 0F 22 0F # D♭mM7: E + C + D♭?
     38:  289 | 03 31 26 22 1E # C*/D♭
     39:  283 | 03 22 05 22 1E # D♭M7(9): E + C + D♭ + E♭?
     40:  290 | 03 22 1E 22 1E # (int) D♭1+8
     41:    5 | 03 31 26 22 1E # C*/D♭: D♭ + E♭ + C + D♭?
     42:  278 | 03 23 0A 22 1E # E♭m7/D♭: D♭ + C + D♭ + E♭ + F♯
     43:  290 | 03 31 26 22 1E # C*/D♭: D♭ + C + D♭ + E♭
     44:  293 | 03 23 0A 35 1E # E♭m7/G: G + C + D♭ + E♭ + F♯
     45:  293 | 03 31 27 35 1E # C*/G: G + C + D♭ + F♯
     46:  871 | 03 31 28 44 1E # C*/F♯: F♯ + C + D♭ + G
     47: 1142 | 03 31 27 44 1E # C*/F♯: F♯ + C + D♭ + F♯
     48:  558 | 03 22 0F 44 1E # D♭mM7/F♯: F♯ + C + D♭ + E
     49:  334 | 03 44 00 44 00 # (int) F♯
     50:    5 | 03 31 26 44 1E # C*/F♯: F♯ + C + D♭ + E♭
     51:  342 | 03 31 22 44 1E # F♯ bass: F♯ + B + C + D♭
     52:  491 | 03 32 16 32 16 # D7(9): F♯ + C + D + E
     53: 1153 | 03 36 00 36 00 # (int) A
     54:    4 | 03 36 0E 36 0E # Am7(11): A + C + D + E
     55:  290 | 03 31 04 31 04 # C(9): G + C + D + E
     56:  218 | 03 32 16 32 16 # D7(9): F♯ + C + D + E
     57:  115 | 03 32 0D 32 0D # Dm(7(9): F + C + D + E
     58:  242 | 03 33 00 33 00 # (int)
     59:    4 | 03 31 2A 33 1E # C*/E: E + C + D + E
     60:   99 | 03 32 29 23 1E # (int) E♭ + C + D
     61:    4 | 03 31 2A 23 1E # C*/E♭: E♭ + C + D + E
     62:  120 | 03 33 00 33 00 # (int) E
     63:    9 | 03 31 2A 33 1E # C*/E
     64:  614 | 03 35 02 35 02 # GM7: G + F♯ + ?
     65:  581 | 03 33 26 35 1E # E*/G: G + E + F + (G?)
     66:  569 | 03 23 2A 35 1E # E♭*/G: G + E♭ + F + (G?)
     67: 1183 | 03 35 00 35 00 # G: (G + G + B)?
     68:  546 | 03 35 29 35 29 # G*: G + F + G♯
     69:  655 | 03 35 00 35 00 # (int) G
     70:    5 | 03 44 22 44 22 # F♯x: G + F♯ + G♯